In [2]:
using JuMP
using Plots
gr()

Plots.GRBackend()

In [28]:
num_time_steps = 10
timespan = 2
dt = timespan / num_time_steps
goal = 0
wall_position = 1
initial_position = -5
initial_velocity = 20
u_limit = 10
wall_spring_stiffness = 1000
wall_spring_damping = 100
q_limit = 10

model = Model()

@variables model begin
    -q_limit <= q[1:num_time_steps] <= q_limit
    q1[1:num_time_steps]
    q2[1:num_time_steps]
    v[1:num_time_steps]
    -u_limit <= u[1:num_time_steps] <= u_limit
    contactforce[1:num_time_steps]
    dampingforce[1:num_time_steps]
    incollision[1:num_time_steps], Bin
end

# Dynamics
@constraint(model, [i = 1:(num_time_steps - 1)], v[i + 1] == v[i] + dt * (u[i + 1] + contactforce[i + 1]))
@constraint(model, [i = 1:(num_time_steps - 1)], q[i + 1] == q[i] + dt * v[i + 1])

# Contact constraints
@constraints model begin
    contactforce .== -wall_spring_stiffness .* q1
    q .== q1 .+ q2 .+ wall_position
    q1 .>= 0
    q1 .<= 10 .* incollision
    q2 .<= 0
    q2 .>= -10 .* (1 .- incollision)
end

# Boundary conditions
@constraints model begin
    q[1] == initial_position
    v[1] == initial_velocity
    q[end] == goal
    v[end] == 0
end

@objective model Min sum(u[i]^2 for i=1:num_time_steps)

solve(model)

ts = linspace(0, timespan, num_time_steps)
q = getvalue.(q)
v = getvalue.(v)
u = getvalue.(u)
q1 = getvalue.(q1)
q2 = getvalue.(q2)
contactforce = getvalue.(contactforce)
incollision = getvalue.(incollision)
q

Optimize a model with 82 rows, 80 columns and 177 nonzeros
Model has 10 quadratic objective terms
Variable types: 70 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+03]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 72 rows and 63 columns
Presolve time: 0.00s
Presolved: 10 rows, 17 columns, 42 nonzeros
Presolved model has 9 quadratic objective terms
Variable types: 15 continuous, 2 integer (2 binary)

Root relaxation: objective 1.255586e+01, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   12.55586    0    1          -   12.55586      -     -    0s
H    0     0                      39.7961965   12.55586  68.4%     -    0s
     0     0   12.55586    0    1   39.79620   12.55586  68.4%     -    0s
  

10-element Array{Float64,1}:
 -5.0     
 -0.987999
  1.04945 
  1.05077 
  0.901295
  0.661244
  0.390855
  0.150363
 -0.0     
 -0.0     

In [21]:
q1

10-element Array{Float64,1}:
 -0.0      
 -0.0      
  0.0494495
  0.0507744
  0.0      
  0.0      
 -0.0      
 -0.0      
 -0.0      
 -0.0      

In [22]:
incollision

10-element Array{Float64,1}:
  0.0
  0.0
  1.0
  1.0
 -0.0
 -0.0
  0.0
  0.0
  0.0
  0.0

In [23]:
plot(ts, q, label="q")
plot!(ts, q1, label="q1")
plot!(ts, q2, label="q1")
plot!(ts, v, label="v")
plot!(ts, u, label="u")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 q 
 
 
 
 q1 
 
 
 
 q1 
 
 
 
 v 
 
 
 
 u

In [24]:
plot(ts, contactforce)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 -50 
 
 
 -40 
 
 
 -30 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 
 
 
 
 y1